In [1]:
import numpy as np
import open3d as o3d
import copy
import pickle
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from matplotlib.colors import LinearSegmentedColormap

from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.IFSelect import IFSelect_RetDone
from OCC.Core.TopAbs import TopAbs_WIRE, TopAbs_EDGE, TopAbs_VERTEX
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopoDS import topods
from OCC.Core.BRep import BRep_Tool
from OCC.Core.BRepAdaptor import BRepAdaptor_Curve
from OCC.Core.GeomAbs import GeomAbs_Circle, GeomAbs_Ellipse, GeomAbs_Line, GeomAbs_BSplineCurve, GeomAbs_Hyperbola, GeomAbs_Parabola
from OCC.Core.TopAbs import TopAbs_WIRE, TopAbs_EDGE, TopAbs_VERTEX
from OCC.Display.SimpleGui import init_display
from OCC.Core.BRepOffsetAPI import BRepOffsetAPI_MakeOffset
from OCC.Core.Quantity import Quantity_Color, Quantity_TOC_RGB
from OCC.Core.gp import gp_Pnt
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeVertex
from PyQt5 import QtWidgets
from OCC.Core.GCPnts import GCPnts_UniformDeflection
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop
from scipy.spatial import ConvexHull

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Read cad model data (STEP file)
width = 0.5

step_file_path = '/home/chris/Code/PointClouds/data/other_files/MortenPartSTEPVersion.STEP'

def load_step_file(file_path):
    step_reader = STEPControl_Reader()
    status = step_reader.ReadFile(file_path)
    if status == IFSelect_RetDone:
        step_reader.TransferRoots()
        shape = step_reader.OneShape()
        return shape
    else:
        raise Exception("Error: Cannot read STEP file.")

shape = load_step_file(step_file_path)

print('Step file has been loaded successfully.')

Step file has been loaded successfully.


In [3]:
def extract_top_edges(shape, deflection=0.1):
    edges = []
    max_z = -float('inf')
    edge_z_coordinates = []

    # First pass to collect the highest Z-coordinate of each edge
    exp_edge = TopExp_Explorer(shape, TopAbs_EDGE)
    while exp_edge.More():
        edge = topods.Edge(exp_edge.Current())
        edge_max_z = -float('inf')
        exp_vertex = TopExp_Explorer(edge, TopAbs_VERTEX)
        vertices = []
        while exp_vertex.More():
            vertex = topods.Vertex(exp_vertex.Current())
            point = BRep_Tool.Pnt(vertex)
            vertices.append(point)
            if point.Z() > edge_max_z:
                edge_max_z = point.Z()
            exp_vertex.Next()
        edge_z_coordinates.append((edge, edge_max_z, vertices))
        if edge_max_z > max_z:
            max_z = edge_max_z
        exp_edge.Next()

    #print(f"Total number of edges: {len(edge_z_coordinates)}")
    #print(f"Maximum Z-coordinate found: {max_z}")

    # Collect edges on the top plane
    top_edges = []
    point_cloud = []
    for edge, z, vertices in edge_z_coordinates:
        if all(abs(vertex.Z() - max_z) < 1e-3 for vertex in vertices):  # Ensure all vertices are at the top surface
            top_edges.append(edge)
            #print(f"Top edge added with highest vertex Z: {z}")

            # Sample points along the edge
            curve = BRepAdaptor_Curve(edge)
            u_min, u_max = curve.FirstParameter(), curve.LastParameter()

            # Calculate the length of the edge
            linear_props = GProp_GProps()
            brepgprop.LinearProperties(edge, linear_props)
            length = linear_props.Mass()

            # Calculate the number of samples based on the length and the desired point density
            num_samples = int(length / deflection)

            for i in range(num_samples):
                u = u_min + i * (u_max - u_min) / (num_samples - 1)
                pnt = curve.Value(u)
                point_cloud.append((pnt.X(), pnt.Y(), pnt.Z()))  # Store the coordinates as a tuple

    print(f"Total number of top view edges: {len(top_edges)}")
    print(f"Total number of points in the cad point cloud: {len(point_cloud)}")
    return top_edges, point_cloud

# Visualize the top edges
def visualize_top_edges(top_edges):
    display, start_display, add_menu, add_function_to_menu = init_display()

    # Assign the color red to each top edge
    red_color = Quantity_Color(1.0, 0.0, 0.0, Quantity_TOC_RGB)
    
    # Display each top edge with the red color
    for edge in top_edges:
        display.DisplayShape(edge, update=True, color=red_color)

    start_display()


# Main execution
print('Generating top view edges and sampling to create the associated point cloud...')
top_edges, cad_points = extract_top_edges(shape)
# if top_edges:
#     visualize_top_edges(top_edges)
# else:
#     print("Top edges not found.")

cad_pcd = o3d.geometry.PointCloud()
cad_pcd.points = o3d.utility.Vector3dVector(np.asarray(cad_points))
o3d.visualization.draw_geometries([cad_pcd])

print('Top view cad point cloud created.')

Generating top view edges and sampling to create the associated point cloud...
Total number of top view edges: 586
Total number of points in the cad point cloud: 81774
Top view cad point cloud created.


In [4]:
# Apply clustering to the cad_pcd

eps = 0.50
min_points = 10

cad_shapes_labels = np.array(cad_pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))

max_label = cad_shapes_labels.max()
print(f"Cad point cloud has {max_label + 1} clusters")
base_cmap = plt.get_cmap("tab20")
color_cycle = [base_cmap(i % 20) for i in range(max_label + 1)]
colors = np.array(color_cycle)[cad_shapes_labels]
colors[cad_shapes_labels<0] = 0
cad_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

cad_points_dictionary = {}
cad_centers_dictionary = {}

# Extract the points and calculate the centers for each cluster
for label in np.unique(cad_shapes_labels):
    if label != -1:  # Ignore noise
        cluster_points = np.asarray(cad_pcd.points)[cad_shapes_labels == label]
        cad_points_dictionary[label] = cluster_points
        cad_centers_dictionary[label] = np.mean(cluster_points, axis=0)

cad_centers_pcd = o3d.geometry.PointCloud()
cad_centers_points = list(cad_centers_dictionary.values())
cad_centers_pcd.points = o3d.utility.Vector3dVector(np.array(cad_centers_points))
cad_centers_pcd.paint_uniform_color([0, 1, 0])  # Color centers blue

o3d.visualization.draw_geometries([cad_pcd])

Precompute neighbors.[========================================] 100%
Clustering[========================================] 100%
Cad point cloud has 79 clusters


In [7]:
import scipy
from scipy.spatial import ConvexHull, Delaunay

def find_interior_point(points):
    """
    Find an interior point of the shape using the convex hull.
    
    Args:
    - points (numpy.ndarray): The points of the shape.
    
    Returns:
    - numpy.ndarray: An interior point of the shape.
    """
    try:
        hull = ConvexHull(points)
        hull_points = points[hull.vertices]
        interior_point = np.mean(hull_points, axis=0)
    except (scipy.spatial.qhull.QhullError, ValueError):
        # Fall back to the centroid if the convex hull fails
        interior_point = np.mean(points, axis=0)
    return interior_point

def magnify_shape(points, interior_point, distance):
    """
    Magnify the shape by moving each point away from the interior point.
    
    Args:
    - points (numpy.ndarray): The points of the shape.
    - interior_point (numpy.ndarray): The interior point of the shape.
    - distance (float): The distance by which to magnify the shape.
    
    Returns:
    - numpy.ndarray: The magnified points.
    """
    vectors = points - interior_point
    magnified_points = points + (vectors / np.linalg.norm(vectors, axis=1, keepdims=True)) * distance
    return magnified_points

# Example usage with your cad_points_dictionary
scale_factor = 1.5  # Define your scale factor here

magnified_points_dict = {}
for label, points in cad_points_dictionary.items():
    interior_point = find_interior_point(points)
    magnified_points = magnify_shape(points, interior_point, scale_factor)
    magnified_points_dict[label] = magnified_points

# Convert the original and magnified points back to a combined point cloud for visualization
all_original_points = np.vstack(list(cad_points_dictionary.values()))
all_magnified_points = np.vstack(list(magnified_points_dict.values()))

combined_points = np.vstack((all_original_points, all_magnified_points))

combined_pcd = o3d.geometry.PointCloud()
combined_pcd.points = o3d.utility.Vector3dVector(combined_points)

# Assign different colors to original and magnified points for better visualization
original_colors = np.tile([0, 0, 1], (all_original_points.shape[0], 1))  # Blue for original points
magnified_colors = np.tile([1, 0, 0], (all_magnified_points.shape[0], 1))  # Red for magnified points

combined_colors = np.vstack((original_colors, magnified_colors))
combined_pcd.colors = o3d.utility.Vector3dVector(combined_colors)

# Visualize the combined shapes
o3d.visualization.draw_geometries([combined_pcd])


/tmp/ipykernel_979961/3606754200.py:18: DeprecationWarning: Please import `QhullError` from the `scipy.spatial` namespace; the `scipy.spatial.qhull` namespace is deprecated and will be removed in SciPy 2.0.0.
  except (scipy.spatial.qhull.QhullError, ValueError):
